# Analisis de los resultados del procesamiento
### Proceso implementado en este notebook:
1. Importar librerias y definir variables
2. Lectura de los archivos binarios que contienen el procesamiento de los sujetos
3. 

In [17]:
import pickle
import os
import xarray as xr
import numpy as np

from src.utils.read_data_from_bids import read_four_subjects
from src.utils.plots.hfo_plots import *
from src.utils.evaluation.evaluation import *
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

processed_data_directory = "data/processed"
results_directory = "data/results"

#argumentos descriptores de la base de datos
kwargs_bids_algoritms_algoritms = {
    'dataset':'data/bids_algoritms_algoritms/ds_algoritms_algoritms004100',
    'datatype':'ieeg',
    'task':'interictal',
    'acquisition':'seeg',
    'run':'01'
} 

In [19]:
files = os.listdir(processed_data_directory)

datasets = []
for archivo in files:
    if archivo.endswith(".pickle"):
        ruta_archivo = os.path.join(processed_data_directory, archivo)

        with open(ruta_archivo, "rb") as file:
            data = pickle.load(file)
    datasets.append(data)

df_process = get_dataframe_results(datasets)

In [15]:
# Crea un diccionario vacío para almacenar los datos
data = {}

# Define las coordenadas vacías
data['statisicians'] = []
data['algorithm_params'] = []

# Crea un objeto Dataset vacío con las coordenadas especificadas
ds = xr.Dataset(data)

#asignacion de coordenadas
ds = ds.assign_coords(statisicians = ['hrr', 'outcome'])

subject_dataset = datasets[0]
for element in subject_dataset:
    new_param = element
    ds = ds.assign_coords(algorithm_params=ds['algorithm_params'].values.tolist() + [new_param])



In [20]:
for subject_dataset in datasets:
    
    subject_id = subject_dataset.attrs['his_id']
    raw, channels = read_four_subjects([subject_id.replace('sub-', '')] ,**kwargs_bids)
    
    hrr_list = []
    outcomes = []

    for element in subject_dataset:
        
        df = subject_dataset[element].to_pandas().T.reset_index()
        df = df.drop(df[df['colors'].isna()].index)
        for bad in raw[0].info['bads']:
            df = df.drop(df[df['ch_split'] == bad].index)    

        condicion = df['status'].isin(['resect', 'resect,soz'])
        filtrado = df[condicion]
        contadores = filtrado['status'].value_counts()

        resection_size = contadores.sum()
        resection_size = round(resection_size/2) 
        
        hfo_region = df.sort_values('counts', ascending=False).head(resection_size)
        condicion = hfo_region['status'].isin(['resect', 'resect,soz'])
        filtrado = hfo_region[condicion]
        contadores = filtrado['status'].value_counts()

        hfo_in_resection = contadores.sum()

        #hrr = round((hfo_in_resection/resection_size),2) + np.random.random() * 0.1 - 0.05#hfo resection ratio
        hrr = round((hfo_in_resection/resection_size),2)
        hrr_list.append(hrr)
                
        if subject_dataset.attrs['outcome'] == 'S':
            outcomes.append(1)
        else:
            outcomes.append(0)

    fusion = list(zip(hrr_list, outcomes))
    fusion_lista = [list(fila) for fila in fusion]

    ds[subject_dataset.attrs['his_id']] = xr.DataArray(fusion_lista, dims=('algorithm_params', 'statisicians'))


/home/ferna96/Documents/tesis/CODETESIS/src/utils/read_data_from_bids.py:62: RuntimeWarning: Did not find any events.tsv associated with sub-HUP160_ses-presurgery_task-interictal_acq-seeg_run-01.

The search_str was "data/bids/ds004100/sub-HUP160/**/ieeg/sub-HUP160_ses-presurgery*events.tsv"
  raws.append(read_raw_bids(bids_path=value, verbose=False, extra_params = extra_params))
/home/ferna96/Documents/tesis/CODETESIS/src/utils/read_data_from_bids.py:62: RuntimeWarning: Did not find any events.tsv associated with sub-HUP190_ses-presurgery_task-interictal_acq-seeg_run-01.

The search_str was "data/bids/ds004100/sub-HUP190/**/ieeg/sub-HUP190_ses-presurgery*events.tsv"
  raws.append(read_raw_bids(bids_path=value, verbose=False, extra_params = extra_params))
/home/ferna96/Documents/tesis/CODETESIS/src/utils/read_data_from_bids.py:62: RuntimeWarning: Did not find any events.tsv associated with sub-HUP138_ses-presurgery_task-interictal_acq-seeg_run-01.

The search_str was "data/bids/ds00410

In [22]:
df_decode_detect = pd.DataFrame({'name':[], 'combination':[]})

for i, element in enumerate(ds.coords["algorithm_params"].values):
    df = ds.sel(algorithm_params = element).to_pandas().T
    df_drop = df.drop("algorithm_params")
    
    df_process = df_process.join(df_drop['hrr']).rename(columns={'hrr': f'hfo_detect_C{i+1}'})
    df_decode_detect = df_decode_detect.append({'name': f'hfo_detect_C{i+1}', 'combination': element}, ignore_index=True)

print(df_decode_detect)

/tmp/ipykernel_11791/2070891058.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_decode_detect = df_decode_detect.append({'name': f'hfo_detect_C{i+1}', 'combination': element}, ignore_index=True)
/tmp/ipykernel_11791/2070891058.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_decode_detect = df_decode_detect.append({'name': f'hfo_detect_C{i+1}', 'combination': element}, ignore_index=True)
/tmp/ipykernel_11791/2070891058.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_decode_detect = df_decode_detect.append({'name': f'hfo_detect_C{i+1}', 'combination': element}, ignore_index=True)
/tmp/ipykernel_11791/2070891058.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a

              name           combination
0    hfo_detect_C1    bw-(40, 249)_ww-50
1    hfo_detect_C2    bw-(40, 249)_ww-70
2    hfo_detect_C3    bw-(40, 249)_ww-90
3    hfo_detect_C4   bw-(40, 249)_ww-110
4    hfo_detect_C5   bw-(40, 249)_ww-130
5    hfo_detect_C6   bw-(40, 249)_ww-150
6    hfo_detect_C7    bw-(40, 100)_ww-50
7    hfo_detect_C8    bw-(40, 100)_ww-70
8    hfo_detect_C9    bw-(40, 100)_ww-90
9   hfo_detect_C10   bw-(40, 100)_ww-110
10  hfo_detect_C11   bw-(40, 100)_ww-130
11  hfo_detect_C12   bw-(40, 100)_ww-150
12  hfo_detect_C13    bw-(70, 130)_ww-50
13  hfo_detect_C14    bw-(70, 130)_ww-70
14  hfo_detect_C15    bw-(70, 130)_ww-90
15  hfo_detect_C16   bw-(70, 130)_ww-110
16  hfo_detect_C17   bw-(70, 130)_ww-130
17  hfo_detect_C18   bw-(70, 130)_ww-150
18  hfo_detect_C19   bw-(100, 160)_ww-50
19  hfo_detect_C20   bw-(100, 160)_ww-70
20  hfo_detect_C21   bw-(100, 160)_ww-90
21  hfo_detect_C22  bw-(100, 160)_ww-110
22  hfo_detect_C23  bw-(100, 160)_ww-130
23  hfo_detect_C

/tmp/ipykernel_11791/2070891058.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_decode_detect = df_decode_detect.append({'name': f'hfo_detect_C{i+1}', 'combination': element}, ignore_index=True)
/tmp/ipykernel_11791/2070891058.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_decode_detect = df_decode_detect.append({'name': f'hfo_detect_C{i+1}', 'combination': element}, ignore_index=True)
/tmp/ipykernel_11791/2070891058.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_decode_detect = df_decode_detect.append({'name': f'hfo_detect_C{i+1}', 'combination': element}, ignore_index=True)
/tmp/ipykernel_11791/2070891058.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a

In [30]:
(ds)

<xarray.Dataset>
Dimensions:           (statisicians: 2, algorithm_params: 42)
Coordinates:
  * statisicians      (statisicians) <U7 'hrr' 'outcome'
  * algorithm_params  (algorithm_params) <U20 'bw-(40, 249)_ww-50' ... 'bw-(1...
Data variables: (12/24)
    sub-HUP160        (algorithm_params, statisicians) float64 0.17 1.0 ... 1.0
    sub-HUP190        (algorithm_params, statisicians) float64 0.08 0.0 ... 0.0
    sub-HUP138        (algorithm_params, statisicians) float64 0.0 0.0 ... 0.0
    sub-HUP187        (algorithm_params, statisicians) float64 0.0 0.0 ... 0.0
    sub-HUP162        (algorithm_params, statisicians) float64 0.0 0.0 ... 0.0
    sub-HUP146        (algorithm_params, statisicians) float64 0.25 1.0 ... 1.0
    ...                ...
    sub-HUP177        (algorithm_params, statisicians) float64 0.12 1.0 ... 1.0
    sub-HUP163        (algorithm_params, statisicians) float64 0.0 1.0 ... 1.0
    sub-HUP180        (algorithm_params, statisicians) float64 0.0 1.0 ... 1.0
    sub-HUP157        (algorithm_params, statisicians) float64 0.0 1.0 ... 1.0
    sub-HUP164        (algorithm_params, statisicians) float64 0.0 1.0 ... 1.0
    sub-HUP150        (algorithm_params, statisicians) float64 nan 1.0 ... 1.0